# How to analyse metabolic networks with Maud

This document explains how to use [Maud](https://github.com/biosustain/Maud/) to fit Bayesian statistical models of steady-state metabolic networks, and how to investigate the results.

## An example Maud input

To start with we'll look through the files in one of the example datasets we have provided. 

You can find a discussion of the input data format in Maud's documentation [here](https://maud-metabolic-models.readthedocs.io/en/latest/usage/inputting.html), but this guide should be enough to get started.

First let's check out the contents of the folder `data/in/linear` by running the first cell below.

In [1]:
import os

import arviz as az
import pandas as pd
import maud

from IPython.display import display

In [2]:
!ls data/linear

config.toml             kinetic_model.toml      priors.csv
experiments.csv         linear.json             true_params_linear.json


The most important of these files, and the only one whose name is important, is `config.toml`. This file tells Maud where to look for the other files it needs, and stores some configuration information that is passed on to Stan. 

Here is what our example config file looks like:

In [3]:
!cat data/linear/config.toml

name = "linear"
kinetic_model = "kinetic_model.toml"
priors = "priors.csv"
experiments = "experiments.csv"
likelihood = true

[cmdstanpy_config]
iter_warmup = 40
iter_sampling = 40
max_treedepth = 8  # for speed in demo: you probably don't want to set this irl!
chains = 4
save_warmup = true
show_progress = false

[ode_config]
abs_tol_forward = 1e-4
rel_tol_forward = 1e-4
abs_tol_backward = 1e-4
rel_tol_backward = 1e-4
abs_tol_quadrature = 1e-4
rel_tol_quadrature = 1e-4
max_num_steps = 1e6
timepoint = 1e3



The first few lines of this file define top-level fields. These tell Maud where to look for the files it needs, and do some high level configuration (`name` picks a name for the input and `likelihood` tells Maud whether or not to run in priors-only mode).

The second section defines a table called `cmdstanpy_config`. Each field here specifies a keyword argument to [cmdstanpy's `sample` method](https://cmdstanpy.readthedocs.io/en/v0.9.77/api.html#cmdstanpy.CmdStanModel.sample). In this case we tell Stan to run 4 markov chains, each of which should use 200 iterations in both its warmup and sampling phases, and to save both warmup and sampling draws in its output csv files.

The fields in the final section specify tuning parameters for the Stan function [`ode_adjoint_tol_ctl`](https://mc-stan.org/docs/2_27/functions-reference/functions-ode-solver.html#adjoint-sensitivity-solver), except for `timepoint`, which tells Maud how long to simulate this function for.

Now let's look at the other files in the folder, starting with `kinetic_model.toml`.

In [4]:
!cat data/linear/kinetic_model.toml

###### Kinetic model ######
[[compartments]]
id = 'c'
name = 'cytosol'
volume = 1

[[compartments]]
id = 'e'
name = 'external'
volume = 1

[[metabolites]]
id = 'M1'
name = 'External metabolite number 1'
balanced = false
compartment = 'e'

[[metabolites]]
id = 'M2'
name = 'External metabolite number 2'
balanced = false
compartment = 'e'

[[metabolites]]
id = 'M1'
name = 'Metabolite number 1'
balanced = true
compartment = 'c'

[[metabolites]]
id = 'M2'
name = 'Metabolite number 2'
balanced = true
compartment = 'c'

[[reactions]]
id = 'r1'
name = 'reaction number 1'
stoichiometry = { M1_e = -1, M1_c = 1}
mechanism = "reversible_modular_rate_law"
[[reactions.enzymes]]
id = 'r1'
name = 'the enzyme that catalyses reaction r1'
[[reactions.enzymes.modifiers]]
modifier_type = 'allosteric_activator'
mic_id = 'M2_c'

[[reactions]]
id = 'r2'
name = 'reaction number 2'
stoichiometry = { M1_c = -1, M2_c = 1 }
mechanism = "reversible_modular_rate_law"
[[reactions.enzymes]]
id = 'r2'
name = 'the enzym

This file consists of a table of `compartments`, a table of `metabolites` and a table of `reactions`.

Next let's have a look at the model'spriors and measurements.

In [5]:
priors = pd.read_csv("data/linear/priors.csv")
experiments = pd.read_csv("data/linear/experiments.csv")

print("Experiments:")
display(experiments)
print("Priors:")
display(priors)

,measurement_type,target_id,experiment_id,measurement,error_scale
0,mic,M1_c,condition_1,0.70,0.10
1,mic,M2_c,condition_1,0.40,0.10
2,mic,M1_e,condition_1,0.50,0.10
3,mic,M2_e,condition_1,0.10,0.10
4,flux,r3,condition_1,0.29,0.10
5,enzyme,r1,condition_1,1.00,0.05
6,enzyme,r2,condition_1,1.00,0.05
7,enzyme,r3,condition_1,1.00,0.05
8,mic,M1_c,condition_2,0.50,0.10
9,mic,M2_c,condition_2,0.40,0.10


,parameter_type,metabolite_id,mic_id,enzyme_id,drain_id,experiment_id,location,scale,pct1,pct99
0,kcat,NaN,NaN,r1,NaN,NaN,1.0,0.60,NaN,NaN
1,kcat,NaN,NaN,r2,NaN,NaN,1.0,0.60,NaN,NaN
2,kcat,NaN,NaN,r3,NaN,NaN,1.0,0.60,NaN,NaN
3,km,NaN,M1_e,r1,NaN,NaN,0.2,0.60,NaN,NaN
4,km,NaN,M1_c,r1,NaN,NaN,0.2,0.60,NaN,NaN
5,km,NaN,M1_c,r2,NaN,NaN,0.2,0.60,NaN,NaN
6,km,NaN,M2_c,r2,NaN,NaN,0.2,0.60,NaN,NaN
7,km,NaN,M2_c,r3,NaN,NaN,0.2,0.60,NaN,NaN
8,km,NaN,M2_e,r3,NaN,NaN,0.2,0.60,NaN,NaN
9,conc_unbalanced,NaN,M1_e,NaN,NaN,condition_1,0.5,0.20,NaN,NaN


## Checking how our model behaves at the initial parameter values

Rather than going straight ahead and sampling, we usually prefer to run a single, fixed-parameter HMC iteration at the initial parameter values and inspect the results before proceeding futher, using the command `maud simulate`. By default Maud initialises at the prior mean, but initial values can also be specified.

The main reason for running `maud simulate` before sampling is to quickly catch cases where the sampler has difficulty traversing the posterior distribution early in the run. This might be due to an error in the provided input data, in which case it is quite likely that the printed output will be weird in a way that makes it easier to track down the problem. Alternatively, it could be that the prior mean happens to be near a natural saddle point or other tricky part of the posterior distribution. In this case it might be necessary to specify custom initial values.

In [6]:
!maud simulate data/linear

Creating output directory: ./maud_output_sim-linear-20210916150852
Copying user input from data/linear to ./maud_output_sim-linear-20210916150852/user_input
INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:compiled model file: /Users/tedgro/Code/Maud/src/maud/model
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:finish chain 1

Simulated concentrations and fluxes:
experiments  mics
condition_1  M1_c    0.295426
             M1_e    0.500000
             M2_c    0.185275
             M2_e    0.100000
condition_2  M1_c    0.340680
             M1_e    0.600000
             M2_c    0.198961
             M2_e    0.100000
Name: conc, dtype: float64
experiments  reactions
condition_1  r1           0.017573
             r2           0.017573
             r3           0.017573
condition_2  r1           0.019833
             r2           0.019833
             r3           0.019833
Name: flux, dtype: float64
experiments  enzymes
condition_1  r1         0.1
             r2         

This command did a few things: it created an output directory in starting with `maud_output_sim-linear`, fetched data from our input folder, ran one iteration of Stan's sampler in fixed-parameter mode and printed some results.

Looking at the steady state metabolite concentrations, fluxes and enzyme concentrations, we can see that none of them are wildly implausible. This is nice, as starting the sampler at a very unrealistic point can lead to poor performance.

However, the simulated concentrations are fairly far away from the measurements, as indicated by the low values of some of the `log-lik-conc` variables.

If we want to inspect the simulation output more closely, we can do so by examining the files in the output folder.

## Generating posterior draws

Since the initial conditions looked ok, it seems safe to start sampling. This can be done with the command `maud sample`, as below.

This command will create a new output directory starting with `maud_output-linear-`, then fill it up with Stan output, then print some diagnostic information and put the results in a structured form using the library [Arviz](https://arviz-devs.github.io/arviz/index.html).

Since, even for this small system, the sampling step is quite slow, you might like to kick this command off in a separate terminal, and skip ahead. Ask how to do this with Jupyterhub if you don't know already!

In [7]:
!maud sample data/linear

Creating output directory: ./maud_output-linear-20210916150855
Copying user input from data/linear to ./maud_output-linear-20210916150855/user_input
INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:compiled model file: /Users/tedgro/Code/Maud/src/maud/model
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:start chain 2
INFO:cmdstanpy:start chain 3
INFO:cmdstanpy:start chain 4
INFO:cmdstanpy:finish chain 2
INFO:cmdstanpy:finish chain 4
INFO:cmdstanpy:finish chain 1
INFO:cmdstanpy:finish chain 3
INFO:cmdstanpy:Processing csv files: /Users/tedgro/dtu/projects/maud_demo/maud_output-linear-20210916150855/samples/model-202109161508-1.csvWarning: non-fatal error reading adaptation data
, /Users/tedgro/dtu/projects/maud_demo/maud_output-linear-20210916150855/samples/model-202109161508-2.csvWarning: non-fatal error reading adaptation data
, /Users/tedgro/dtu/projects/maud_demo/maud_output-linear-20210916150855/samples/model-202109161508-3.csvWarning: non-fatal error reading adapta

## Analysing the run



In [8]:
output_dir = "data/example_output_linear/"  # change this if you have your own output

infd = az.from_netcdf(os.path.join(output_dir,"infd.nc"))

infd

Inference data with groups:
	> posterior
	> sample_stats

Warmup iterations saved (warmup_*).

## Making out-of-sample predictions